In [1]:
import pandas as pd
import numpy as np
import win32com.client
import time

In [47]:
file = "C:/Users\lukec\OneDrive - University of Cambridge\Projects\PhD\Data\IHS\Ethylene\ETHYLENE FROM WIDE RANGE NAPHTHA, MAXIMUM ETHYLENE, FRONT END DEMETHANIZER - Germany.xlsx"
footer = 11
headerLines = 5
colSep = 6

In [52]:
def refresh_excel_file(file:str, pause=False):
    # Refresh Excel file to have numbers for formula cells
    xlapp = win32com.client.DispatchEx("Excel.Application")
    wb = xlapp.Workbooks.Open(file)
    wb.RefreshAll()
    if pause: time.sleep(pause)
    wb.Save()
    wb.Close()
    xlapp.Quit()
    return file

In [55]:
def excel_multilayer_table_extract(df, dataType='Attribute', header=0, colStart=0, colEnd=-1):
    data = df[header:][df.columns[colStart:colEnd]]
    data.columns = [dataType] + list(data.iloc[0][1:].values)
    data.columns = list(map(lambda x: 'Unit' if x is np.nan else x, data.columns.values))
    data.dropna(subset=[data.columns[0]], inplace=True)

    typesInd = [0] + list(np.where(data.index[1:] - data.index[:-1] != 1)[0]+1)
    data['Type'] = list(map(lambda x: data[dataType].values[typesInd[np.sum(np.array(typesInd) < x+1)-1]], range(len(data))))
    data.dropna(subset=data.columns[1:-1], how='all', inplace=True)
    return data

In [91]:
# Import sheet
sheet = pd.read_excel(refresh_excel_file(file),skipfooter=footer)
materials = excel_multilayer_table_extract(sheet, dataType='Material', header=headerLines, colEnd=colSep)
cost = excel_multilayer_table_extract(sheet, dataType='Cost', header=headerLines, colStart=colSep+1)
properties = list(map(lambda x: x.strip(), sheet.loc[2][0].split(",")))
properties = dict(map(lambda x: x.split(": "), properties))

In [92]:
properties

{'Geography': 'Germany',
 'Product': 'ETHYLENE',
 'PEP Cost Index': '1087',
 'Price': '135.83 ¢/KG',
 'Code': '2M-1622',
 'Location Factor': '1.06 (1US $ = 0.85)',
 'Data version': '2021 Q3',
 'Research Year': '2014'}

In [93]:
materials

,Material,Unit Cost,Unit,Unit Consumption,Unit,Product Cost,Type
8,RAW MATERIALS,NaN,NaN,NaN,NaN,210.157112,RAW MATERIALS
9,CATALYST AND CHEMICALS,NaN,NaN,NaN,NaN,1.051605,RAW MATERIALS
10,NAPHTHA,66.790014,¢/KG,3.13079,TONNE,209.105507,RAW MATERIALS
12,BY-PRODUCT CREDITS,NaN,NaN,NaN,NaN,-197.790217,BY-PRODUCT CREDITS
13,C4 FRACTION8,80.424633,¢/KG,-0.34879,TONNE,-28.051308,BY-PRODUCT CREDITS
14,FUEL GAS,6.563179,¢/MMCAL,-6994.67782,MMCAL,-45.90732,BY-PRODUCT CREDITS
15,FUEL OIL.,3.963287,¢/MMCAL,-1720.15038,MMCAL,-6.81745,BY-PRODUCT CREDITS
16,HYDROGEN (95 VOL%),21.276446,¢/NM3,-386.88141,NM3,-8.231461,BY-PRODUCT CREDITS
17,"PROPYLENE, POLYMER GRADE",135.738615,¢/KG,-0.51044,TONNE,-69.286419,BY-PRODUCT CREDITS
18,PYROLYSIS GASOLINE,71.968401,¢/KG,-0.5488,TONNE,-39.496259,BY-PRODUCT CREDITS


In [94]:
cost

,Cost,Capacity,NaN,Type
8,"INVESTMENT, US$ MILLION",1870.061319,2954.851395,"INVESTMENT, US$ MILLION"
9,BATTERY LIMITS,1280.105202,2022.671987,"INVESTMENT, US$ MILLION"
10,OFF SITES,589.956117,932.179409,"INVESTMENT, US$ MILLION"
13,RAW MATERIALS,210.157112,210.157112,"PRODUCTION COSTS, US ¢/KG"
14,BY PRODUCT CREDITS,-197.790217,-197.790217,"PRODUCTION COSTS, US ¢/KG"
15,UTILITIES,44.776985,44.776985,"PRODUCTION COSTS, US ¢/KG"
16,VARIABLE COSTS,57.143881,57.143881,"PRODUCTION COSTS, US ¢/KG"
17,MAINTENANCE MATERIALS,6.143454,4.853582,"PRODUCTION COSTS, US ¢/KG"
18,OPERATING LABOR (11.0/SHIFT),1.502188,0.751094,"PRODUCTION COSTS, US ¢/KG"
19,OPERATING SUPPLIES,0.150219,0.075109,"PRODUCTION COSTS, US ¢/KG"
